[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/atmos-cloud-sim-uj/PySDM.git/master?filepath=PySDM_examples/Berry_1967_Figs_5_8_10/demo.ipynb)    
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/atmos-cloud-sim-uj/PySDM/blob/master/PySDM_examples/Berry_1967_Figs_5_8_10/demo.ipynb)

#### based on Figs. 5, 8, & 10 from Berry 1967 (J. Atmos. Sci. 24)  "_Cloud Droplet Growth by Collection_"   
https://doi.org/10.1175/1520-0469(1967)024%3C0688:CDGBC%3E2.0.CO;2

In [1]:
"""
clone and install PySDM dependencies in Colab
(to use GPU on Colab set hardware accelerator to 'GPU' before session start
in the "Runtime :: Change runtime type :: Hardware accelerator" menu)
"""
import os, sys
if 'google.colab' in sys.modules:
    %cd /content
    if not os.path.isdir('PySDM'):
        !git clone --depth 1 https://github.com/atmos-cloud-sim-uj/PySDM.git
        %cd PySDM
    else:
        %cd PySDM
        !git pull
    !pip --quiet install --requirement requirements.txt
    !ldconfig

In [2]:
import os, sys
if 'google.colab' in sys.modules:
    %cd /content/PySDM
else:
    sys.path.insert(0, os.path.join(os.getcwd(), '../..'))

In [3]:
from numpy import errstate
from PySDM.backends import CPU, GPU
from PySDM.dynamics.coalescence.kernels import Geometric, Hydrodynamic, Electric
from PySDM_examples.Berry_1967_Figs_5_8_10.spectrum_plotter import SpectrumPlotter
from PySDM_examples.Berry_1967_Figs_5_8_10.settings import Settings
from PySDM_examples.Shima_et_al_2009_Fig_2.example import run
from PySDM_examples.utils import widgets

In [4]:
progbar = widgets.IntProgress(min=0, max=100, description='%')
class ProgbarUpdater:
    def __init__(self, progbar, max_steps):
        self.max_steps = max_steps
        self.steps = 0
        self.progbar = progbar

    def notify(self):
        self.steps += 1
        self.progbar.value = 100 * (self.steps / self.max_steps)
        
class WidgetsFreezer:
    def __init__(self, widgets):
        self.widgets = widgets
        
    def observe(*args, **kwargs):
        pass
    
    @property
    def value(self):
        return self
    
    def __enter__(self):
        for widget in self.widgets:
            widget.disabled = True
        return self
            
    def __exit__(self, *args, **kwargs):
        for widget in self.widgets:
            widget.disabled = False

In [5]:
def demo(*, freezer, n_SD, n_step, n_plot, kernel, adaptive, smooth, gpu):
    with freezer:
        with errstate(all='raise'):
            settings = Settings()
            backend = GPU if gpu else CPU
            settings.n_sd = 2 ** n_SD
            settings.adaptive = adaptive
            settings.dt = 10 if adaptive else settings.dt
            settings._steps = [i * (n_step // n_plot) for i in range(n_plot + 1)]
            if kernel == 'geometric sweep-out':
                settings.kernel = Geometric()
            elif kernel == 'electric field 3000V/cm':
                settings.kernel = Electric()
            else:
                settings.kernel = Hydrodynamic()

            states, _ = run(settings, backend, (ProgbarUpdater(progbar, settings.steps[-1]),))

        with errstate(invalid='ignore'):
            plotter = SpectrumPlotter(settings)
            plotter.smooth = smooth
            for step, state in states.items():
                plotter.plot(state, step * settings.dt)
            plotter.show()

In [6]:
n_SD = widgets.IntSlider(value=14, min=12, max=18, step=1, description='$log_2(n_{SD})$', continuous_update=False)
n_step = widgets.IntSlider(value=1000, step=100, min=0, max=2400, description='$n_{step}$', continuous_update=False)
n_plot = widgets.IntSlider(value=10, step=1, min=1, max=16, description='$n_{plot}$', continuous_update=False)
sliders = widgets.HBox([n_SD, n_step, n_plot])

adaptive = widgets.Checkbox(value=False, description='adaptive dt')
smooth = widgets.Checkbox(value=True, description='smooth plot')
gpu = widgets.Checkbox(value=False, description='GPU')
options = [adaptive, smooth]
if GPU.ENABLE:
    options.append(gpu)
kernel = widgets.Select(
    options=['geometric sweep-out', 'electric field 3000V/cm', 'hydrodynamic capture'],
    value='geometric sweep-out',
    description='kernel:',
    rows=3
)
options.append(kernel)
boxes = widgets.HBox(options)
    
freezer = WidgetsFreezer([n_SD, n_step, n_plot, kernel, adaptive, gpu])
self = widgets.interactive_output(demo, 
                                  {'freezer': freezer, 'n_SD': n_SD, 'n_step': n_step, 'n_plot': n_plot, 
                                   'kernel': kernel, 'adaptive': adaptive, 'smooth': smooth, 'gpu': gpu})

display(sliders, boxes, progbar, self)

IntProgress(value=100, description='%')

Output()